<a href="https://colab.research.google.com/github/letun1987/Test-task-Database-manager-/blob/main/%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_%D1%87%D0%B0%D1%81%D1%82%D1%8C_2_%D0%A4%D0%B8%D0%BD%D0%B0%D0%BB_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import tqdm
from google.colab import files


In [ ]:
df = pd.read_excel('Test_Python.xlsx', sheet_name="TEST")

In [ ]:
df.head()

,Id,Название,Web,Телефон,Адрес,ИНН,Описание (О компании)
0,e82e90ae-4dcf-4c80-895d-4a7b81bc9593,Производственная компания Цементный завод Семей,semeycement.com,NaN,NaN,NaN,NaN
1,f351c419-43e5-4940-bbcd-850462f5a4a6,Протей Технологии,tl.protei.ru,NaN,NaN,NaN,NaN
2,ed3b8aee-d1c4-4698-948c-063e49872465,Ютэйр-Вертолетные Услуги,heli.utair.ru,NaN,NaN,NaN,NaN
3,f9bc736b-6740-48b2-97bd-fcf0807cd256,Гварда,gwarda.ru,NaN,NaN,NaN,NaN
4,eefbc460-f538-402e-baec-d72d3b002b74,Гематэк,gematech.ru,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     100 non-null    object 
 1   Название               100 non-null    object 
 2   Web                    100 non-null    object 
 3   Телефон                0 non-null      float64
 4   Адрес                  0 non-null      float64
 5   ИНН                    0 non-null      float64
 6   Описание (О компании)  0 non-null      float64
dtypes: float64(4), object(3)
memory usage: 5.6+ KB


In [ ]:
"""
напишем простой цикл, который поможет сразу выявить проблемные сайты из списка.
Статус 200 - это все хорошо, ответ приходит, иные статусы это уже какие-то проблемы,
а проблемы лучше сразу отложить в сторону.
"""
for i in df['Web']:
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
  try:
    res = requests.get(f"https://{i}", headers=headers, timeout=20)
  except:
    pass
  if res.status_code != 200:
    print(i, res.status_code)
  else:
    pass

marilend.by 403


marilend.by 403 - сайт открывается через браузер, и это очень похоже на то, что у них стоит защита от парсеров. Их можно оставить, но лучше удалить, чтобы они создавали различные исключения и ошибки.


In [ ]:
df = df[(df['Web'] != 'lcpumpliner.com') & (df['Web'] != 'chinakangrui.com') &
                (df['Web'] != 'marilend.by')]

In [ ]:
def phone (row):
  try:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    res = requests.get(f"https://{row}/", headers=headers, timeout=20)
    if res.status_code == 200:
      print(row)
      soup = BeautifulSoup(res.text, "html.parser")
      info = soup.find('body').text
      text = info.replace('\n', '').replace('\r', '').replace('\t', '').lower()
      list_of_phone = re.findall(r"[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]", text)#.str.replace(' ', '')
      #print(i, list_of_phone)
      #return list_of_phone
      if len(list_of_phone) != 0:
        print(row, list_of_phone)
        return list_of_phone
      elif len(list_of_phone) == 0:
        list_of_phone = re.findall(r"(?:(?:8|\+7)[\- ]?)?(?:\(?\d{3}\)?[\- ]?)?[\d\- ]{7,10}", text)
        print(row, list_of_phone)
        return list_of_phone
      else:
        pass
    else:
      print(row, 'Страница не отвечает')
  except:
    pass

df['Телефон'] = df['Web'].apply(phone)

semeycement.com
semeycement.com ['61 285 932', '31108-2020', '33174-2014', '22266-2013', '308 (7222) 31-52-36', '8 (7222) 31-67-04', '8 701 098 25 38', '8 707 038 79 78', '8 777 817 88 87']
tl.protei.ru
tl.protei.ru ['+7 (812) 401-63-25']
heli.utair.ru
heli.utair.ru ['+7 3452 29-8855', '+7 800 234-8855', '+7 3452 29-8855', '+7 800 234-8855', '+7 3452 29-8855', '+7 800 234-8855']
gwarda.ru
gwarda.ru ['+7 (499) 653-90-13', '+7 (499) 653-90-13', '+7 (499) 653-90-13']
gematech.ru
gematech.ru ['+7 911 291-45-32', '6312195878', '1196313020441', '+7 911 291-44-30', '+7 911 291-44-30', '+7 911 291-44-30', '+7 911 291-44-30', '+7 911 291-44-30', '+7 911 291-44-30', '+7 911 291-45-32', '+7 812 389-35-22', '+7 911 291-44-30', '+7 812 389-35-223314532']
pskgeodor.ru
pskgeodor.ru ['8 800 775 30 93', '8-800-775-3093', '14-17.04.2023', '15.02.2022', '4834-043-75957906-2013', '8 800 775 30 93']
asia-techno.com
asia-techno.com []
hqts.ru
hqts.ru ['8 (495) 713 93 48', '9709001040', '1177746485498', '8 (

In [ ]:
percent_missing = df['Телефон'].isnull().sum() * 100 / len(df)
print(f"Процент пропущенных значений в столбце Телефон: {round(percent_missing)}%")

Процент пропущенных значений в столбце Телефон: 34%


In [ ]:
df[df['Телефон'].notna()]

,Id,Название,Web,Телефон,Адрес,ИНН,Описание (О компании)
0,e82e90ae-4dcf-4c80-895d-4a7b81bc9593,Производственная компания Цементный завод Семей,semeycement.com,"[61 285 932, 31108-2020, 33174-2014, 22266-201...",NaN,NaN,NaN
1,f351c419-43e5-4940-bbcd-850462f5a4a6,Протей Технологии,tl.protei.ru,[+7 (812) 401-63-25],NaN,NaN,NaN
2,ed3b8aee-d1c4-4698-948c-063e49872465,Ютэйр-Вертолетные Услуги,heli.utair.ru,"[+7 3452 29-8855, +7 800 234-8855, +7 3452 29-...",NaN,NaN,NaN
3,f9bc736b-6740-48b2-97bd-fcf0807cd256,Гварда,gwarda.ru,"[+7 (499) 653-90-13, +7 (499) 653-90-13, +7 (4...",NaN,NaN,NaN
4,eefbc460-f538-402e-baec-d72d3b002b74,Гематэк,gematech.ru,"[+7 911 291-45-32, 6312195878, 1196313020441, ...",NaN,NaN,NaN
...,...,...,...,...,...,...,...
93,71759255-e903-4df4-b392-dcecae6ea39e,Нефтесервис Инжиниринг,ns-ing.ru,"[+7 (495) 120-18-98 0, +7 (495) 120-18-98, +8...",NaN,NaN,NaN
94,d68ee2d4-1a29-495b-a7d7-dbecc6d9ad53,Роснефтегазинструмент,rngi.ru,"[+7 (861) 212-12-05, +7 (918) 088-44-22, +7 (8...",NaN,NaN,NaN
96,c249024f-0893-4c26-8c65-da14462adf17,Пьезус,piezus.ru,"[+7 (495) 796-92-20, +7 (495) 796-92-208 800 7...",NaN,NaN,NaN
98,df900e88-669d-4b61-ae97-d83a3fa31b36,Shandong Wintech Technology Co. Ltd,sdwintech.en.made-in-china.com,[600.00-250],NaN,NaN,NaN


In [ ]:
df.replace(to_replace=[None], value=np.nan, inplace=True)

In [ ]:
"""
удалям из столбца с телефонами все лишние знаки
"""
def clear_phone(row):
  if type(row) == float:
    pass
  else:
    list_of_number = []
    for a in row:
      item = re.sub('\D', '', a)
      list_of_number.append(item)
    return list_of_number


df['Телефон'] = df['Телефон'].apply(clear_phone)
df.head()

,Id,Название,Web,Телефон,Адрес,ИНН,Описание (О компании)
0,e82e90ae-4dcf-4c80-895d-4a7b81bc9593,Производственная компания Цементный завод Семей,semeycement.com,"[61285932, 311082020, 331742014, 222662013, 30...",NaN,NaN,NaN
1,f351c419-43e5-4940-bbcd-850462f5a4a6,Протей Технологии,tl.protei.ru,[78124016325],NaN,NaN,NaN
2,ed3b8aee-d1c4-4698-948c-063e49872465,Ютэйр-Вертолетные Услуги,heli.utair.ru,"[73452298855, 78002348855, 73452298855, 780023...",NaN,NaN,NaN
3,f9bc736b-6740-48b2-97bd-fcf0807cd256,Гварда,gwarda.ru,"[74996539013, 74996539013, 74996539013]",NaN,NaN,NaN
4,eefbc460-f538-402e-baec-d72d3b002b74,Гематэк,gematech.ru,"[79112914532, 6312195878, 1196313020441, 79112...",NaN,NaN,NaN


Ниже видно, что помимо телефонов, в список попадают и другие цифры которые находятся на странице. В дальнейшем это все необходимо очищать и удалять дубликаты.

In [ ]:
for i in df['Телефон'].iloc[10:20]:
  print(i)

None
['74952666700', '79282892486', '79614271545', '61131394', '74952666700', '79282892486', '79614271545']
['73472163826', '20230922', '20231003', '20230911', '20230911', '73472163826']
['78313049444', '150022012', '15092023', '11092023', '18082023', '10082023', '25072023', '12072023', '620062023', '19062023', '14062023', '24052023', '23052023', '24042023', '20042023', '17042023', '78313049444']
['78314205220', '22052023', '16022023', '17012023', '202226102022', '202201092022', '324082022', '202217082022', '310802092022', '202229042022', '78314205220', '78314205220']
['369036', '15000', '2023', '369036', '2023']
['73432280314', '73432280314', '73432280314', '73432280314', '73432280317', '73436357100', '73436357102']
['88002508355', '88002508355', '88002508355', '7341256651688002508355', '1840039742', '115184000524088002508355', '27072006', '73412566516', '88002508355', '4080120180200240280300340400', '1357101520253035404550556065707580859095100110120', '1351015202530354045505560657075

In [ ]:
"""
Данная функция собирает ищет на странице слово ИНН и собирает следующие 20 символов,
которые идут после этого слова. 20 - это чтобы точно все числа из номера ИНН вошли
в отбор, т.к. отступы могут отличаться. В контексте данной функции мы обращаеся у URL
адреса к папке(пути) contacts.
"""
def INN (row):
  try:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    res = requests.get(f"https://{row}/contacts/", headers=headers, timeout=20)
    if res.status_code == 200:
      soup = BeautifulSoup(res.text, "html.parser")
      info = soup.find('body').text
      info.replace('\n', '').replace('\r', '').replace('\t', '')
      index = info.find("ИНН")
      if index == -1:
        return np.nan
      else:
        return info[index:index+20]
    else:
      pass
  except:
    pass


df['ИНН'] = df['Web'].apply(INN)
df.head(5)

,Id,Название,Web,Телефон,Адрес,ИНН,Описание (О компании)
0,e82e90ae-4dcf-4c80-895d-4a7b81bc9593,Производственная компания Цементный завод Семей,semeycement.com,"[61285932, 311082020, 331742014, 222662013, 30...",NaN,NaN,NaN
1,f351c419-43e5-4940-bbcd-850462f5a4a6,Протей Технологии,tl.protei.ru,[78124016325],NaN,ИНН: 7802862459\nОГРН,NaN
2,ed3b8aee-d1c4-4698-948c-063e49872465,Ютэйр-Вертолетные Услуги,heli.utair.ru,"[73452298855, 78002348855, 73452298855, 780023...",NaN,None,NaN
3,f9bc736b-6740-48b2-97bd-fcf0807cd256,Гварда,gwarda.ru,"[74996539013, 74996539013, 74996539013]",NaN,None,NaN
4,eefbc460-f538-402e-baec-d72d3b002b74,Гематэк,gematech.ru,"[79112914532, 6312195878, 1196313020441, 79112...",NaN,None,NaN


In [ ]:
df[df['ИНН'].notna()]

,Id,Название,Web,Телефон,Адрес,ИНН,Описание (О компании)
1,f351c419-43e5-4940-bbcd-850462f5a4a6,Протей Технологии,tl.protei.ru,[78124016325],NaN,ИНН: 7802862459\nОГРН,NaN
8,bd9d8be6-8464-4e52-85a7-9527b17a6414,Hqts — Quality Control Company,hqts.ru,"[84957139348, 9709001040, 1177746485498, 84957...",NaN,ИНН: 9709001040 ОГРН,NaN
49,ac7b25bb-6e79-42eb-abe6-f5d416033822,Фотополимер,fotopolymer.ru,"[74956496646, 79175499595, 74956499999, 749564...",NaN,ИНН\n9727008009\n\n\nРас,NaN
53,84610a8f-beae-493e-b02e-f110326ceaac,Русские Вездеходы Пластун,tpu-plastun.ru,"[88008008080, 88005112136, 1217800042283, 7801...",NaN,ИНН 7801696576\nЮриди,NaN
78,246e0b37-d0ba-4c4a-b245-e43912540f22,Изосистема,isosystem.ru,"[8929707024588462150468, 892970702458846215046...",NaN,ИНН 6319702090\nКПП 6,NaN
96,c249024f-0893-4c26-8c65-da14462adf17,Пьезус,piezus.ru,"[74957969220, 7495796922088007077526]",NaN,ИНН — 7722857693\nКПП,NaN


Данных очень мало, основная проблема связана с тем, что у кого-то информация об ИНН хранится на главной странице, а кого-то в папке contacts, также встречаются следующие названия папок: contact без S, kontakty и т.д. Ниже я написал функцию, которая собирает все ссылки с главной страницы каждого сайта и группирует их в словарь, где значения это сколько раз подобная ссылка (она не совсем ссылка, а часть адреса, если конкретно папка) встречалась на всех сайтах из списка, а ключ это папка(часть URL сайта)

In [ ]:
percent_missing = df['ИНН'].isnull().sum() * 100 / len(df)
print(f"Процент пропущенных значений в столбце Телефон: {round(percent_missing)}%")

Процент пропущенных значений в столбце Телефон: 94%


In [ ]:
"""
создаем словарь со всеми сслыками с главных страниц сайта, данный подход
помогает найти общие закономерности в структурах сайтов. Так как каталоги(папки)
в URL адресе страницы могут иметь разные названия, например: contact, contactS,
kontakty.
"""
dict_catalog = {}
count = 1
for i in df['Web']:
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
  try:
    reqs = requests.get(f"https://{i}/", headers=headers, timeout=5)
  except:
    pass
  if reqs.status_code == 200:
    content = reqs.text
    soup = BeautifulSoup(content, 'html.parser')
    for h in soup.findAll('li'):
      a = h.find('a')
      try:
        if 'href' in a.attrs:
          url = a.get('href')
          if len(url)<2:
            pass
          else:
            if url not in dict_catalog.keys():
              dict_catalog[url] = count
            else:
              dict_catalog[url] += 1
      except:
	      pass

sorted_values = sorted(dict_catalog.items(), key=lambda kv: kv[1], reverse=True)
sorted_values[:20]

[('/about/', 36),
 ('/contacts/', 31),
 ('/news/', 22),
 ('/uslugi/', 21),
 ('/case/', 20),
 ('javascript:void(0);', 17),
 ('/service/', 17),
 ('/catalog/', 16),
 ('/statii/', 16),
 ('/advantage/technical-advantages/', 16),
 ('/contacts', 15),
 ('/product/', 14),
 ('https://www.bellowvalves.com/download.html', 14),
 ('/news', 12),
 ('https://avtotransformator.ru/quality_system/', 12),
 ('https://aisnn.com/author/admin/', 12),
 ('https://intrascada.com/ru/about-us-ru/', 12),
 ('/catalog/teploizolyatsiya/', 12),
 ('/catalog/teploizolyatsiya/teploizolyatsiya_dlya_trub/', 12),
 ('/catalog/gidroizolyatsiya/', 12)]

В результате работы словаря мы получили данные, что 37 сайтов в своем URL содержат папку contacts, а 36 папку about.

Собрав самые часто встречающиеся названия, можно создавать сложенные функции и циклы, которые все это дело будут перебирать и вытаксивать оттуда информацию по максимуму, тем самым минимизируя пропуски.

Ниже код который позволяет извлекать информацию о компании.

In [ ]:
"""
При помощи данной функции получаем информацию о компании, для поиска используется
соотвествующий раздел на сайте about/. Как правило описание компании состоит из
нескольких длинных разделов(абзацев). Чтобы лишние строки не попадали в результирующую
выдачи применяется фильтр - брать, только то что длиннее 70 символов, конечно это можно,
как сократить так и увеличить.
"""
def info_about_company (row):
  try:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    res = requests.get(f"https://{row}/about/", headers=headers, timeout=20)
    print(row, res.status_code)
    if res.status_code == 200:
      content = res.text
      soup = BeautifulSoup(content, 'html.parser')#from_encoding="iso-8859-8"
      for script in soup(["script", "style"]):
        script.extract()

      text = soup.body.get_text(separator='\n', strip=True)
      lines = (line.strip() for line in text.splitlines())
      chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
      text = '\n'.join(chunk for chunk in chunks if chunk)
      #print(text)

      list = text.split('\n')
      clear_list = []
      for i in list:
        if bool(re.search('[\u0400-\u04FF]', i)) == True and len(i)>70:
          clear_list.append(i)
        else:
          pass
      return clear_list
    else:
      pass
  except:
    pass


df['Описание (О компании)'] = df['Web'].apply(info_about_company)
df.head(5)

semeycement.com 200
tl.protei.ru 404
heli.utair.ru 404
gwarda.ru 404
gematech.ru 404
pskgeodor.ru 404
asia-techno.com 404
hqts.ru 404
avtotransformator.ru 404
reallab.ru 200
aviatron-ufa.ru 200
gtlab.pro 404
aisnn.com 200
bolt57.ru 404
kmz-ural.com 404
z-magnum.ru 404
steelimpex.ru 404
armas71.ru 200
severtrucks.ru 200
systeme.ru 200
таграс-рс.рф 404
new.jereh.com/ru 403
macromer.ru 200
ipc-bugulma.com 200
simecs.ru 404
ooosvs.ru 404
ecoresurs.ru 200
lcpumpliner.com 200
univis.ru 200
cells-trade.ru 404
wiremeshbst.com 404
joyo-mc.com 404
cyfintube.com 404
salessolution.ru 404
thermoelectrika.com 404
sunmoonhose.net 404
eni-bbmv.ru 404
marilend.by 403
fotopolymer.ru 200
jichai.com 404
tpu-plastun.ru 200
optimaengineparts.com 404
bellowvalves.com 404
slc-jh.ru 404
ural-project.ru 404
unchemsol.com 200
himagregat-info.ru 200
сферанефтьигаз.рф 404
park-stroy.ru 200
dgrit.ru 404
intrascada.com 200
тмс-групп.рф 404
unigreen-energy.com 403
tormin-lighting.com 404
isosystem.ru 404
cz-ex.com 20

,Id,Название,Web,Телефон,Адрес,ИНН,Описание (О компании)
0,e82e90ae-4dcf-4c80-895d-4a7b81bc9593,Производственная компания Цементный завод Семей,semeycement.com,"[61285932, 311082020, 331742014, 222662013, 30...",NaN,NaN,[История Цементного завода в г. Семей началось...
1,f351c419-43e5-4940-bbcd-850462f5a4a6,Протей Технологии,tl.protei.ru,[78124016325],NaN,ИНН: 7802862459\nОГРН,None
2,ed3b8aee-d1c4-4698-948c-063e49872465,Ютэйр-Вертолетные Услуги,heli.utair.ru,"[73452298855, 78002348855, 73452298855, 780023...",NaN,None,None
3,f9bc736b-6740-48b2-97bd-fcf0807cd256,Гварда,gwarda.ru,"[74996539013, 74996539013, 74996539013]",NaN,None,None
4,eefbc460-f538-402e-baec-d72d3b002b74,Гематэк,gematech.ru,"[79112914532, 6312195878, 1196313020441, 79112...",NaN,None,None


In [ ]:
df[df['Описание (О компании)'].notna()]

,Id,Название,Web,Телефон,Адрес,ИНН,Описание (О компании)
0,e82e90ae-4dcf-4c80-895d-4a7b81bc9593,Производственная компания Цементный завод Семей,semeycement.com,"[61285932, 311082020, 331742014, 222662013, 30...",NaN,NaN,[История Цементного завода в г. Семей началось...
11,418bd326-cba5-4dd6-b9e1-89866e1a11bf,Reallab!,reallab.ru,"[74952666700, 79282892486, 79614271545, 611313...",NaN,NaN,[стория компании насчитывает 30 лет и начинает...
12,74ed6e9e-ff53-451c-be2c-807001411ca3,Авиатрон,aviatron-ufa.ru,"[73472163826, 20230922, 20231003, 20230911, 20...",NaN,None,"[– российская сервисная компания, оказывает ст..."
14,b4381bc8-274e-4554-9e5c-022a9455fb0b,Акустические Измерительные Системы - НН АИС-НН,aisnn.com,"[78314205220, 22052023, 16022023, 17012023, 20...",NaN,None,[СЖУ-1-МВ-К - микроволновый рефлекс-радарный д...
21,0e5b7876-e666-46d5-a8fe-aa5906e81342,Армас,armas71.ru,"[74872525645, 332592015, 332592015, 332592015,...",NaN,None,[]
22,4deac00f-8086-4ebc-a00a-a74857f567cf,Север,severtrucks.ru,"[88004443481, 88007075572, 88004443481]",NaN,NaN,[Основатель и директор компании – Андрей Евген...
24,cbbac268-1028-4c32-a801-7cbba6c4a439,СИСТЭМ ЭЛЕКТРИК,systeme.ru,[8800200644684957779988],NaN,NaN,[Систэм Электрик — российская компания с миров...
27,61aecf2a-fb3b-4d0f-99a6-7230d2db439f,Макромер-Интеграл-Авто,macromer.ru,"[78002006595, 78002006595, 78002006595, 780020...",NaN,None,[В военные годы отец Владимира Степановича - С...
28,f4fa6533-df03-4d32-a15a-510949f93c83,ИПЦ,ipc-bugulma.com,"[78005515955, 88005515955, 77172695915]",NaN,NaN,[Проведение научно-исследовательских и опытно–...
32,3de87227-8861-4d5b-a49a-10cf2aef6dd6,Экоресурс,ecoresurs.ru,"[4732334623, 4732727820, 4732462858]",NaN,None,[Российский разработчик и производитель оборуд...


In [ ]:
df['Описание (О компании)'].iloc[32] # описание ecoresurs.ru

['Российский разработчик и производитель оборудования промышленной автоматизации',
 'Техника компании успешно используется на крупнейших предприятиях нефтеперерабатывающей, нефтехимической, химической, энергетической, добывающей, металлургической, целлюлозно-бумажной и пищевой промышленности России.',
 'Решения и сервисы, предоставляемые компанией, позволяют нашим заказчикам максимально снижать издержки, увеличивать прибыль и повышать эффективность производства по всей цепочке создания стоимости.',
 'Основной профиль деятельности компании — разработка и серийное производство программно-технических средств и систем противоаварийной защиты, регистрации, сигнализации, регулирования и управления технологическими процессами на основе контроллеров с зарегистрированным товарным знаком БАЗИС®.',
 'Компания выпускает контроллеры как во взрывозащищенном исполнении с маркировками [Exia]IIC, 0ExiaIICT6, так и в общепромышленном исполнении.',
 'Контроллеры имеют соответствующие сертификаты и разреш

In [ ]:
df['Описание (О компании)'].iloc[49] # описание fotopolymer.ru

['ООО «Научно-производственное объединение «Фотополимер» на\xa0сегодняшний день единственная российская компания\xa0— производитель фотополимерных материалов торговых знаков «PHOSMAT» и\xa0«ФотоФикс», отверждаемых за\xa0несколько минут ультрафиолетом невидимого спектра излучения с\xa0длиной волны 365\xa0Нм (лучи солнца, UV-прожектор «PHOSMAT 400» собственной разработки), а\xa0также за\xa0несколько секунд с\xa0помощью лазера синего светодиода.',
 'Разработана собственная технология производства гибких фотополимерных труб, фотополимерных материалов промышленного и\xa0потребительского назначения. Производство реализовано на\xa0собственной производственной базе (г. Солнечногорск, МО). В\xa0состав производственного комплекса входит композитная лаборатория. 95\xa0процентов сырья\xa0— это отечественные материалы. Оставшиеся пять процентов возможно синтезировать в\xa0России.',
 'НПО "Фотополимер" выиграло конкурс Министерства промышленности России в номинации "Лидер в композитном импортоза- ме

In [ ]:
df['Описание (О компании)'].iloc[94] # описание fotopolymer.ru

['Клапаны обратные шаровые (противовыбросовые) КОШЗ-Р1 для бурильных труб',
 'Клапаны обратные шаровые (противовыбросовые) КОШТ-Р1 для колонны НКТ гибких труб',
 'Краны шаровые для колонны бурильных труб с замковой резьбой типа КШЗ-Р1',
 'Краны шаровые полнопроходные КШ-Р1 для трубопроводов (манифольдов) высокого давления',
 'Головка цементировочная для бурильных труб со съемным коллектором ГЦК-БТ-Р1',
 'Головка цементировочная двухпробочная со съемным коллектором для бурильных труб ГЦК2-БТ-Р1',
 'Головка цементировочная универсальная для цементирования с вращением с ВСП ГЦУ-Р3',
 'Головка цементировочная универсальная для цементирования с вращением с верхним силовым приводом (ВСП) ГЦУ2-Р3',
 'Головка цементировочная для бурильных труб для цементирования с вращением с ВСП ГЦУ-БТ-Р3',
 'Насос опрессовочный для стенда СОГ-100Р1 в комплекте с обвязкой НО-Д-Р1',
 'Благодарим Вас за проявленный интерес к нашей продукции. Уже много лет мы специализируемся в производстве первоклассного оборуд

Для получения адреса из текста, я руководствовался следующей логикой. В большинстве случаев слово город пишется сокращенно, как "г.". Данное сокращение фигурирует в адресах всегда, в то время как улица "ул." может отсутствовать, по причине того, что адрес может иметь в названии "проспект" или "бульвар" и т.д. Единственная проблема, что в выдачу могут попасть года "2023 г."

In [ ]:
"""
Функция для поиска на странице шаблона "г.", после нахождения такого шаблона,
забирает соседний текст слева 30 символов и справа 100 символов.
"""
def adress (row):
  try:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    res = requests.get(f"https://{row}", headers=headers, timeout=20)
    print(row, res.status_code)
    if res.status_code == 200:
      content = res.text
      soup = BeautifulSoup(content, 'html.parser')#from_encoding="iso-8859-8"
      for script in soup(["script", "style"]):
        script.extract()

      text = soup.body.get_text(separator='\n', strip=True)
      lines = (line.strip() for line in text.splitlines())
      chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
      text = '\n'.join(chunk for chunk in chunks if chunk)
      #print(text)
      list_1 = []
      for item in re.finditer(r"\sг[\.,]", text):
        list_1.append(text[item.span()[0]-30:item.span()[1]+100])
      if len(list_1)==0:
        return np.nan
      else:
        print(list_1)
        return list_1
    else:
      pass
  except:
    pass


df['Адрес'] = df['Web'].apply(adress)
df.head(5)

semeycement.com 200
['та через реку Иртыш, аэропорта г. Алматы, высокогорного катка Медео и селезащитной плотины в Алматы, Дворца мира и согласия в г. Аста', 'маты, Дворца мира и согласия в г. Астана и комплекса «Байтерек». Завод активно участвует в реализации государственных проектов и явля', 'ек […]\n071412, Казахстан, ВКО, г.Семей, Западный промышленный узел 45\nПн-Пт 8:30-17:30\n8 (7222) 31-52-36 (приемная)\n8 (7222) 31-67-04']
tl.protei.ru 200
['-63-25\nsales@protei.ru\n197342, г. Санкт-Петербург, вн. тер. г. муниципальный округ Ланское, ул. Белоостровская, д. 6 литера А, помещ.', ', г. Санкт-Петербург, вн. тер. г. муниципальный округ Ланское, ул. Белоостровская, д. 6 литера А, помещ. 30-Н, офис 615, БЦ Birch Hou']
heli.utair.ru 200
gwarda.ru 200
['объекту.\nКонтакты\nООО «ГВАРДА»\nг. Липецк, ул. Льва Толстого, стр. 7, пом. 1\nПн.-Пт. 9:00-18:00\n+7 (499) 653-90-13\nАдрес электронной п']
gematech.ru 200
pskgeodor.ru 200
['тового комплекса "Центральный" г. Саратова в с. Сабуровка.\

,Id,Название,Web,Телефон,Адрес,ИНН,Описание (О компании)
0,e82e90ae-4dcf-4c80-895d-4a7b81bc9593,Производственная компания Цементный завод Семей,semeycement.com,"[61285932, 311082020, 331742014, 222662013, 30...","[та через реку Иртыш, аэропорта г. Алматы, выс...",NaN,[История Цементного завода в г. Семей началось...
1,f351c419-43e5-4940-bbcd-850462f5a4a6,Протей Технологии,tl.protei.ru,[78124016325],"[-63-25\nsales@protei.ru\n197342, г. Санкт-Пет...",ИНН: 7802862459\nОГРН,None
2,ed3b8aee-d1c4-4698-948c-063e49872465,Ютэйр-Вертолетные Услуги,heli.utair.ru,"[73452298855, 78002348855, 73452298855, 780023...",NaN,None,None
3,f9bc736b-6740-48b2-97bd-fcf0807cd256,Гварда,gwarda.ru,"[74996539013, 74996539013, 74996539013]","[объекту.\nКонтакты\nООО «ГВАРДА»\nг. Липецк, ...",None,None
4,eefbc460-f538-402e-baec-d72d3b002b74,Гематэк,gematech.ru,"[79112914532, 6312195878, 1196313020441, 79112...",NaN,None,None


In [ ]:
df.head()

,Id,Название,Web,Телефон,Адрес,ИНН,Описание (О компании)
0,e82e90ae-4dcf-4c80-895d-4a7b81bc9593,Производственная компания Цементный завод Семей,semeycement.com,"[61285932, 311082020, 331742014, 222662013, 30...","[та через реку Иртыш, аэропорта г. Алматы, выс...",NaN,[История Цементного завода в г. Семей началось...
1,f351c419-43e5-4940-bbcd-850462f5a4a6,Протей Технологии,tl.protei.ru,[78124016325],"[-63-25\nsales@protei.ru\n197342, г. Санкт-Пет...",ИНН: 7802862459\nОГРН,None
2,ed3b8aee-d1c4-4698-948c-063e49872465,Ютэйр-Вертолетные Услуги,heli.utair.ru,"[73452298855, 78002348855, 73452298855, 780023...",NaN,None,None
3,f9bc736b-6740-48b2-97bd-fcf0807cd256,Гварда,gwarda.ru,"[74996539013, 74996539013, 74996539013]","[объекту.\nКонтакты\nООО «ГВАРДА»\nг. Липецк, ...",None,None
4,eefbc460-f538-402e-baec-d72d3b002b74,Гематэк,gematech.ru,"[79112914532, 6312195878, 1196313020441, 79112...",NaN,None,None


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Id                     100 non-null    object
 1   Название               100 non-null    object
 2   Web                    100 non-null    object
 3   Телефон                66 non-null     object
 4   Адрес                  35 non-null     object
 5   ИНН                    6 non-null      object
 6   Описание (О компании)  23 non-null     object
dtypes: object(7)
memory usage: 5.6+ KB


In [ ]:
"""
Данная функция работает хуже предыдущей, но как дополнительный поиск и на отдельных
сайтах возможно может показывать лучший результат по поиску значений
"""
def adress (row):
  try:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    res = requests.get(f"https://{row}", headers=headers, timeout=20)
    print(row, res.status_code)
    if res.status_code == 200:
      content = res.text
      soup = BeautifulSoup(content, 'html.parser')#from_encoding="iso-8859-8"
      for script in soup(["script", "style"]):
        script.extract()

      text = soup.body.get_text(separator='\n', strip=True)
      lines = (line.strip() for line in text.splitlines())
      chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
      text = '\n'.join(chunk for chunk in chunks if chunk)
      #print(text)

      list = text.split('\n')
      res = []
      for a, i in enumerate(list):
        if i.startswith('г.'):
          print(a, i)
          res.append(list[a-5:a+5])
        else:
          pass
      res = [[ item for item in elt if len(item) > 2 ] for elt in res]
      print(res)
      return res
    else:
      pass
  except:
    pass


df['Адрес'] = df['Web'].apply(adress)
df.head(5)

In [ ]:
with pd.ExcelWriter('Тестовое_задание_Летуновский_Михаил_часть_2.xlsx') as writer:
    df.to_excel(writer, sheet_name='Table', index=False)


In [ ]:
df.sample(10)

,Id,Название,Web,Телефон,Адрес,ИНН,Описание (О компании)
13,3044af1c-58c9-4bc9-b74c-2a4aa1218549,ГТЛАБ,gtlab.pro,"[78313049444, 150022012, 15092023, 11092023, 1...","[@gtlab.pro\nНижегородская обл., г. Саров, ул....",None,None
77,eb4f5766-4c6f-456a-b24f-e4cfb5ede006,Zhejiang Tormin Electrical Co. Ltd,tormin-lighting.com,"[8657756966366, 8657756966616, 8657756966166, ...",NaN,None,None
55,0d3363ce-b181-4ae7-9bb2-ee574bdb3ab7,Jiangsu Hongyuan Pipe Co. Ltd,jshongju.com,None,None,None,None
12,74ed6e9e-ff53-451c-be2c-807001411ca3,Авиатрон,aviatron-ufa.ru,"[73472163826, 20230922, 20231003, 20230911, 20...","[fa.ru\nПриезжайте к нам в офис:\nг. Уфа, ул. ...",None,"[– российская сервисная компания, оказывает ст..."
7,d93526d8-e712-4745-926b-9bf5c65e6d62,Паллант Инжиниринг,pallant-yar.ru,None,None,None,None
24,cbbac268-1028-4c32-a801-7cbba6c4a439,СИСТЭМ ЭЛЕКТРИК,systeme.ru,[8800200644684957779988],"[ля промышленной автоматизации.\nг. Коммунар, ...",NaN,[Систэм Электрик — российская компания с миров...
79,49e25c04-b322-4aa1-b302-e3e6247549c6,Yangzhou Yuxin Pipe Fittings Co. Ltd,unistarsgroup.com,None,None,None,None
52,dcb077d2-9b16-41aa-a411-f1b0b2b34818,Shandong Weima Equipment Science & Technology ...,weimagroup.com,None,None,None,None
71,f358b77e-3473-4557-8c49-e8062b173a27,Drillgeorobotics,dgrit.ru,"[73832337173, 5403064937]","[\nEmail: dgr@rit-it.com\n630024, г. Новосибир...",None,None
39,a603a811-9b91-4eba-80e6-fb4b19b5150e,Joyo M&C Technology Co. Ltd,joyo-mc.com,"[861065421356, 861065422881, 8613693324568]",NaN,None,None
